In [1]:
import numpy as np
import requests
import pandas as pd
from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image


In [2]:
URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor&version=1.3.0&crs=EPSG:4326&transparent=false&WIDTH={}&HEIGHT={}&BBOX={}&FORMAT=image/tiff&TIME={}"
KM_PER_DEG_AT_EQ = 111.
df = "fire_archive_M6_96619.cvs"

In [3]:
def calculate_width_height(extent, resolution):
    """
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution, i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    lats = extent[::2]
    lons = extent[1::2]
    km_per_deg_at_lat = KM_PER_DEG_AT_EQ * np.cos(np.pi * np.mean(lats) / 180.)
    width = int((lons[1] - lons[0]) * km_per_deg_at_lat / resolution)
    height = int((lats[1] - lats[0]) * KM_PER_DEG_AT_EQ / resolution)
    #print(width, height)
    return (width, height)


In [4]:
def modis_url(time, extent, resolution):
    """
    time: utc time in iso format EG: 2020-02-19T00:00:00Z
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution, i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    width, height = calculate_width_height(extent, resolution)
    extent = ','.join(map(lambda x: str(x), extent))
    return (width, height, URL.format(width, height, extent, time))

In [5]:
width, height, url = modis_url('2019-03-01T00:25:00Z', [-14.50162974683544, 142.71768591772153, -13.50698575949367,144.02234968354432], 0.25)

In [ ]:
for i in range(0,2000):
    print(df.loc[i,'latitude'],df.loc[i,'longitude'],df.loc[i,'acq_date'],df.loc[i,'acq_time'])
    if len(str(df.loc[i,'acq_time'].item()))%2 == 0:
        line = str(df.loc[i,'acq_time'])
    else:
        line='0'+str(df.loc[i,'acq_time'])
        
    if len(line) == 2:
        line='00'+line
    
    a=[line[i:i+2] for i in range(0, len(line), 2)]
    Lyt = df.loc[i,'acq_date']+'T'+a[0]+':'+a[1]+':00Z'
    width, height, url = modis_url(Lyt, [df.loc[i,'latitude']-0.25, df.loc[i,'longitude']-0.25, df.loc[i,'latitude']+0.25, df.loc[i,'longitude']+0.25], 0.25)
    response = requests.get(url)
    img = BytesIO(response.content)
    pil_img = Image.open(img)
    #display(pil_img)
    pil_img=pil_img.save('fire'+str(i)+'.jpg')

In [ ]:
print(url)
response = requests.get(url)
img = BytesIO(response.content)
Image.open(img)

In [ ]:
print(df.shape[0])

183593
